In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# 超参数设置
epochs = 500
batch_size = 64
input_dim = 25
output_dim = 5
layer_dims = [25, 6, 6, 6, 5]
colors = ['#1f77b4', '#ff7f0e', '#d62728', '#2ca02c']  # 蓝、橙、红、绿
num_runs = 3  # 运行次数

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((5, 5)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 归一化到[-1,1]
])

# 加载MNIST数据集（仅取0-4）
train_dataset = datasets.MNIST(root='/tmp/data', train=True, download=True, transform=transform)
train_mask = (train_dataset.targets < 5)
train_dataset.data = train_dataset.data[train_mask]
train_dataset.targets = train_dataset.targets[train_mask]

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 定义神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(layer_dims[0], layer_dims[1]), nn.Sigmoid(),
            nn.Linear(layer_dims[1], layer_dims[2]), nn.Sigmoid(),
            nn.Linear(layer_dims[2], layer_dims[3]), nn.Sigmoid(),
            nn.Linear(layer_dims[3], layer_dims[4])
        )
        
        # 注册钩子捕获中间层输出
        self.activations = {}
        def get_activation(name):
            def hook(model, input, output):
                self.activations[name] = output.detach()
            return hook
        
        self.layers[1].register_forward_hook(get_activation('T1'))
        self.layers[3].register_forward_hook(get_activation('T2'))
        self.layers[5].register_forward_hook(get_activation('T3'))

    def forward(self, x):
        return self.layers(x)

# EI计算函数
def calculate_ei(input_dim, output_dim, layer_func, n_samples=200):
    # 蒙特卡洛采样
    X = torch.rand(n_samples, input_dim) * 2 - 1  # 均匀分布[-1,1]
    
    log_dets = []
    for x in X:
        x = x.unsqueeze(0)  # 增加 batch 维度
        x.requires_grad_(True)  # 启用梯度计算
        
        # 计算输出
        y = layer_func(x)
        
        # 计算雅可比矩阵
        jacobian = torch.zeros((output_dim, input_dim))
        for i in range(output_dim):
            grad = torch.autograd.grad(y[0, i], x, retain_graph=True)[0]
            jacobian[i] = grad.squeeze(0)  # 去除 batch 维度
        
        # 处理非方阵：J @ J^T 的行列式
        det = torch.det(jacobian @ jacobian.T)
        log_dets.append(torch.log(torch.abs(det) + 1e-8).item())
    
    expectation = np.mean(log_dets)
    
    # 噪声估计（使用训练数据）
    X_val = torch.rand(1000, input_dim) * 2 - 1
    y_pred = layer_func(X_val)
    rmse = torch.sqrt(torch.mean((y_pred - y_pred.mean(dim=0))**2, dim=0))
    sigma_term = 0.5 * torch.sum(torch.log(rmse**2 + 1e-8))
    
    # NIS公式
    ei = input_dim * np.log(2) - 0.5 * output_dim * (1 + np.log(2 * np.pi)) - sigma_term + expectation
    return ei.item()

# 多次运行实验
all_ei_history = []
for run in range(num_runs):
    # 初始化模型
    model = Net()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # 训练模型并计算EI
    ei_history = {'X-T1': [], 'T1-T2': [], 'T2-T3': [], 'T3-Y': []}
    for epoch in range(epochs):
        # 训练步骤
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            target_onehot = torch.zeros(output.shape[0], output_dim).scatter_(1, target.unsqueeze(1), 1.0)
            loss = criterion(output, target_onehot)
            loss.backward()
            optimizer.step()
        
        # 每50个epoch计算EI
        if epoch % 50 == 0 or epoch == epochs-1:
            model.eval()
            ei_history['X-T1'].append(calculate_ei(layer_dims[0], layer_dims[1], lambda x: model.layers[1](x)))
            ei_history['T1-T2'].append(calculate_ei(layer_dims[1], layer_dims[2], lambda x: model.layers[3](x)))
            ei_history['T2-T3'].append(calculate_ei(layer_dims[2], layer_dims[3], lambda x: model.layers[5](x)))
            ei_history['T3-Y'].append(calculate_ei(layer_dims[3], layer_dims[4], lambda x: model.layers[7](x)))
    
    # 保存每次运行的结果
    all_ei_history.append(ei_history)



NameError: name 'criterion' is not defined

In [1]:
# 可视化多次运行结果
for idx, label in enumerate(['X-T1', 'T1-T2', 'T2-T3', 'T3-Y']):
    for run in range(num_runs):
        # 只在第一次运行时添加标签，其他运行不添加（避免重复图例）
        if run == 0:
            plt.plot(x_ticks, all_ei_history[run][label], 
                    color=colors[idx], 
                    alpha=0.3 + 0.2 * run, 
                    linewidth=2, 
                    label=label)
        else:
            plt.plot(x_ticks, all_ei_history[run][label],
                    color=colors[idx],
                    alpha=0.3 + 0.2 * run,
                    linewidth=2)
plt.xlabel('Training Epoch', fontsize=12)
plt.ylabel('EI (bits)', fontsize=12)
plt.title('MNIST Layer-wise Effective Information Dynamics', fontsize=14)
plt.legend(loc='upper left', frameon=False)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xlim(0, epochs)
plt.ylim(-20, 40)  # 纵轴区间

plt.tight_layout()
plt.show()

NameError: name 'num_runs' is not defined